# 15-1 시퀀스-투-시퀀스(Sequence-to-Sequence)

Input -> Encoder -> CONTEXT Vector -> Decoder -> Output 의 구조 <br>
encoder, decoder은 각각이 RNN 구조. (LSTM, GRU) <br>
디코더의 초기 입력 = < sos > 최종 출력 = < eos >

In [1]:
import pandas as pd
import shutil
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t') # source, target
del lines['lic']
len(lines)

189986

In [4]:
lines = lines.loc[:, 'src':'tar']
lines = lines[:60000]

In [6]:
lines.sample(10)

,src,tar
48447,I remember the letter.,Je me souviens de cette lettre.
249,Get out.,Décampez !
19903,The bird is dead.,L'oiseau est mort.
11783,Tom is similar.,Tom est pareil.
44101,This is the best one.,Il s’agit du meilleur.
53604,Don't you speak French?,Ne parlez-vous pas français ?
32708,You cannot do this.,Tu ne peux pas faire cela.
37679,Tom is somewhat shy.,Tom est quelque peu timide.
28501,How is your mother?,Comment va votre mère ?
13351,He's a slowpoke.,C'est un lambin.


번역 문장에 해당하는 프랑스어 데이터는 < sos >, < eos > 심볼을 넣어준다 <br>
< sos > -> \t <br>
< eos > -> \n

In [7]:
lines.tar = lines.tar.apply(lambda x: '\t'+x+'\n')
lines.sample(10)

,src,tar
45859,You're overemotional.,\tVous vous laissez trop dominer par vos émoti...
23045,I didn't kiss Tom.,\tJe n'ai pas embrassé Tom.\n
31460,Tom can't use this.,\tTom ne peut pas utiliser ceci.\n
10501,I was upstairs.,\tJ'étais en haut.\n
57399,The day is almost over.,\tLa journée tire à sa fin.\n
14254,I'm a happy man.,\tJe suis un homme heureux.\n
54893,I hope we achieve that.,\tJ'espère que nous finirons cela.\n
27661,Do you accept Visa?,\tVous prenez la VISA ?\n
7368,I was drugged.,\tJ'ai été droguée.\n
27810,Don't make a sound.,\tNe faites pas un bruit !\n


In [9]:
# src 글자 집합
src_vocab = set()
for line in lines.src:
    for char in line:
        src_vocab.add(char)

# tar 글자 집합
tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

print(len(src_vocab)+1) # src_vocab 크기
print(len(tar_vocab)+1) # tar_vocab 크기

79
105


In [12]:
# 정렬
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))

# 인덱스 부여
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])

In [15]:
# 영어 문장 정수 인코딩
encoder_input = []
for line in lines.src:
    temp_X = []
    for w in line:
        temp_X.append(src_to_index[w]) # 글자에 해당하는 정수
    encoder_input.append(temp_X)

In [17]:
# 프랑스어 문장 정수 인코딩
decoder_input = []
for line in lines.tar:
    temp_X = []
    for w in line:
        temp_X.append(tar_to_index[w]) # 글자에 해당하는 정수
    decoder_input.append(temp_X)

디코더의 예측값과 비교하기 위한 실제값을 알려주어야 함.
실제값은 < sos >가 없으므로 정수 인코딩에서 < sos >를 삭제함 (\t)

In [18]:
# <sos> 제거. 맨앞 제거하면 됨
decoder_target = []
for line in lines.tar:
    t = 0
    temp_X = []
    for w in line:
        if t>0:
            temp_X.append(tar_to_index[w])
        t += 1
    decoder_target.append(temp_X)

In [19]:
print(decoder_input[0])
print(decoder_target[0])

[1, 48, 53, 3, 4, 2]
[48, 53, 3, 4, 2]


In [22]:
# 패딩
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])

encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [23]:
# 원핫인코딩
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

## 모델 설계

In [24]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [25]:
# Encoder
encoder_inputs = Input(shape=(None, len(src_vocab)+1))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs) # encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c] # LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

# Decoder
decoder_inputs = Input(shape=(None, len(tar_vocab)+1))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states) # encoder의 state를 받음)

decoder_softmax_layer = Dense(len(tar_vocab)+1, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [26]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2) # 모델 훈련

Epoch 1/50
750/750 [==============================] - 9s 12ms/step - loss: 0.7483 - val_loss: 0.6768
Epoch 2/50
750/750 [==============================] - 8s 11ms/step - loss: 0.4663 - val_loss: 0.5426
Epoch 3/50
750/750 [==============================] - 8s 11ms/step - loss: 0.3912 - val_loss: 0.4799
Epoch 4/50
750/750 [==============================] - 8s 11ms/step - loss: 0.3485 - val_loss: 0.4475
Epoch 5/50
750/750 [==============================] - 8s 11ms/step - loss: 0.3200 - val_loss: 0.4198
Epoch 6/50
750/750 [==============================] - 9s 11ms/step - loss: 0.2988 - val_loss: 0.4020
Epoch 7/50
750/750 [==============================] - 9s 11ms/step - loss: 0.2824 - val_loss: 0.3912
Epoch 8/50
750/750 [==============================] - 9s 11ms/step - loss: 0.2692 - val_loss: 0.3810
Epoch 9/50
750/750 [==============================] - 9s 11ms/step - loss: 0.2582 - val_loss: 0.3753
Epoch 10/50
750/750 [==============================] - 8s 11ms/step - loss: 0.2486 - val_lo

## Seq2seq 기계 번역 동작
1. 번역하고자 하는 입력 문장이 인코더에 들어가서 은닉 상태와 셀 상태를 얻음 <br>
2. 두 상태와 < sos >에 해당하는 '\t'를 디코더로 보냄. <br>
3. 디코더가 < eos >에 해당하는 '\n'이 나올 때까지 다음 문자를 예측하는 행동을 반복함

In [27]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [28]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [33]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    tar_vocab_size = len(tar_vocab)+1
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [34]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Hi.
정답 문장: Salut !
번역기가 번역한 문장: Salut.
-----------------------------------
입력 문장: I see.
정답 문장: Aha.
번역기가 번역한 문장: Je vous en allement.
-----------------------------------
입력 문장: Hug me.
정답 문장: Serrez-moi dans vos bras !
번역기가 번역한 문장: Serre-moi dans tes bras !
-----------------------------------
입력 문장: Hold it!
정답 문장: Restez où vous êtes !
번역기가 번역한 문장: Ne bougez plus !
-----------------------------------
입력 문장: I crashed.
정답 문장: Je me suis écrasée.
번역기가 번역한 문장: Je me suis émoré.
